In [1]:
import pandas as pd
import numpy as np
from nilearn.connectome import ConnectivityMeasure
import pingouin
import pickle
import os
import time

In [2]:
data = pd.read_csv("Phenotypic_V1_0b_preprocessed1.csv")
data.head()

,Unnamed: 0,Unnamed: 0.1,SUB_ID,X,subject,SITE_ID,FILE_ID,DX_GROUP,DSM_IV_TR,AGE_AT_SCAN,...,qc_notes_rater_1,qc_anat_rater_2,qc_anat_notes_rater_2,qc_func_rater_2,qc_func_notes_rater_2,qc_anat_rater_3,qc_anat_notes_rater_3,qc_func_rater_3,qc_func_notes_rater_3,SUB_IN_SMP
0,0,1,50002,1,50002,PITT,no_filename,1,1,16.77,...,NaN,OK,NaN,fail,ic-parietal-cerebellum,OK,NaN,fail,ERROR #24,1
1,1,2,50003,2,50003,PITT,Pitt_0050003,1,1,24.45,...,NaN,OK,NaN,OK,NaN,OK,NaN,OK,NaN,1
2,2,3,50004,3,50004,PITT,Pitt_0050004,1,1,19.09,...,NaN,OK,NaN,OK,NaN,OK,NaN,OK,NaN,1
3,3,4,50005,4,50005,PITT,Pitt_0050005,1,1,13.73,...,NaN,OK,NaN,maybe,ic-parietal-cerebellum,OK,NaN,OK,NaN,0
4,4,5,50006,5,50006,PITT,Pitt_0050006,1,1,13.37,...,NaN,OK,NaN,maybe,ic-parietal slight,OK,NaN,OK,NaN,1


In [3]:
path = "data_roi_time_series/Outputs/ccs/filt_global/rois_aal/"
files = os.listdir(path)

## FIQ

In [4]:
filenames = []
data = data[data['DX_GROUP']==2]
for i in range(len(files)):
    temp = files[i].split('_')
    idx = temp.index('rois')
    name = '_'.join(temp[:idx])
    if name in data['FILE_ID'].values:
        df = data[data['FILE_ID']==name]
        if not (np.isnan(df['FIQ'].values[0])):
            if df['FIQ'].values[0] != -9999:
                filenames.append(name)
    
num_subjects = len(filenames)

442

In [5]:
all_subjects = [[]]*num_subjects
i = 0
for file in filenames:
    f = open(path+file+'_rois_aal.1D','r')
    temp = [line.split() for line in f]
    all_subjects[i] = np.array(temp[1:]).astype('float64')
    i += 1

In [6]:
correlation_measure = ConnectivityMeasure(kind='correlation')
correlation_matrices = correlation_measure.fit_transform(all_subjects)

(442, 116, 116)

In [7]:
p_thresh = 0.01
pos_significant = []
neg_significant = []
pos_pvals = []
neg_pvals = []
for j in range(116):
    for k in range(j,116):
        wt = []
        iq = []
        for i in range(len(all_subjects)):
            wt.append(correlation_matrices[i,j,k])
            fiq = data[data["FILE_ID"]==filenames[i]]['FIQ'].values[0]
            iq.append(fiq)
        c = pingouin.corr(wt,iq,method='bicor')
        pval = c['p-val'].values[0]
        if pval < p_thresh:
            if c['r'].values[0] > 0:
                pos_significant.append((j,k))
                pos_pvals.append(pval)
            else:
                neg_significant.append((j,k))
                neg_pvals.append(pval)
    #print(j,end='.')

0.1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.25.26.27.28.29.30.31.32.33.34.35.36.37.38.39.40.41.42.43.44.45.46.47.48.49.50.51.52.53.54.55.56.57.58.59.60.61.62.63.64.65.66.67.68.69.70.71.72.73.74.75.76.77.78.79.80.81.82.83.84.85.86.87.88.89.90.91.92.93.94.95.96.97.98.99.100.101.102.103.104.105.106.107.108.109.110.111.112.113.114.115.1740.1848816871643


In [8]:
with open('fiq_pos_significant', 'wb') as fp:
    pickle.dump(pos_significant, fp)

with open('fiq_neg_significant', 'wb') as fp:
    pickle.dump(neg_significant, fp)
    
with open('fiq_pos_pvals', 'wb') as fp:
    pickle.dump(pos_pvals, fp)

with open('fiq_neg_pvals', 'wb') as fp:
    pickle.dump(neg_pvals, fp)

## VIQ

In [13]:
filenames = []
data = data[data['DX_GROUP']==2]
for i in range(len(files)):
    temp = files[i].split('_')
    idx = temp.index('rois')
    name = '_'.join(temp[:idx])
    if name in data['FILE_ID'].values:
        df = data[data['FILE_ID']==name]
        if not (np.isnan(df['VIQ'].values[0])):
            if df['VIQ'].values[0] != -9999:
                filenames.append(name)
    
num_subjects = len(filenames)

391

In [14]:
all_subjects = [[]]*num_subjects
i = 0
for file in filenames:
    f = open(path+file+'_rois_aal.1D','r')
    temp = [line.split() for line in f]
    all_subjects[i] = np.array(temp[1:]).astype('float64')
    i += 1

In [15]:
correlation_measure = ConnectivityMeasure(kind='correlation')
correlation_matrices = correlation_measure.fit_transform(all_subjects)
correlation_matrices.shape

(391, 116, 116)

In [16]:
p_thresh = 0.01
pos_significant = []
neg_significant = []
pos_pvals = []
neg_pvals = []
for j in range(116):
    for k in range(j,116):
        wt = []
        iq = []
        for i in range(len(all_subjects)):
            wt.append(correlation_matrices[i,j,k])
            viq = data[data["FILE_ID"]==filenames[i]]['VIQ'].values[0]
            iq.append(viq)
        c = pingouin.corr(wt,iq,method='bicor')
        pval = c['p-val'].values[0]
        if pval < p_thresh:
            if c['r'].values[0] > 0:
                pos_significant.append((j,k))
                pos_pvals.append(pval)
            else:
                neg_significant.append((j,k))
                neg_pvals.append(pval)
    #print(j,end='.')

0.1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.25.26.27.28.29.30.31.32.33.34.35.36.37.38.39.40.41.42.43.44.45.46.47.48.49.50.51.52.53.54.55.56.57.58.59.60.61.62.63.64.65.66.67.68.69.70.71.72.73.74.75.76.77.78.79.80.81.82.83.84.85.86.87.88.89.90.91.92.93.94.95.96.97.98.99.100.101.102.103.104.105.106.107.108.109.110.111.112.113.114.115.1524.4364664554596


In [19]:
with open('viq_pos_significant', 'wb') as fp:
    pickle.dump(pos_significant, fp)

with open('viq_neg_significant', 'wb') as fp:
    pickle.dump(neg_significant, fp)

with open('viq_pos_pvals', 'wb') as fp:
    pickle.dump(pos_pvals, fp)

with open('viq_neg_pvals', 'wb') as fp:
    pickle.dump(neg_pvals, fp)

## PIQ

In [28]:
filenames = []
data = data[data['DX_GROUP']==2]
for i in range(len(files)):
    temp = files[i].split('_')
    idx = temp.index('rois')
    name = '_'.join(temp[:idx])
    if name in data['FILE_ID'].values:
        df = data[data['FILE_ID']==name]
        if not (np.isnan(df['PIQ'].values[0])):
            if df['PIQ'].values[0] != -9999:
                filenames.append(name)
    
num_subjects = len(filenames)

402

In [29]:
all_subjects = [[]]*num_subjects
i = 0
for file in filenames:
    f = open(path+file+'_rois_aal.1D','r')
    temp = [line.split() for line in f]
    all_subjects[i] = np.array(temp[1:]).astype('float64')
    i += 1

In [30]:
correlation_measure = ConnectivityMeasure(kind='correlation')
correlation_matrices = correlation_measure.fit_transform(all_subjects)

(402, 116, 116)

In [32]:
p_thresh = 0.01
pos_significant = []
neg_significant = []
pos_pvals = []
neg_pvals = []
for j in range(116):
    for k in range(j,116):
        wt = []
        iq = []
        for i in range(len(all_subjects)):
            wt.append(correlation_matrices[i,j,k])
            piq = data[data["FILE_ID"]==filenames[i]]['PIQ'].values[0]
            iq.append(piq)
        c = pingouin.corr(wt,iq,method='bicor')
        pval = c['p-val'].values[0]
        if pval < p_thresh:
            if c['r'].values[0] > 0:
                pos_significant.append((j,k))
                pos_pvals.append(pval)
            else:
                neg_significant.append((j,k))
                neg_pvals.append(pval)
    #print(j,end='.')

0.1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.25.26.27.28.29.30.31.32.33.34.35.36.37.38.39.40.41.42.43.44.45.46.47.48.49.50.51.52.53.54.55.56.57.58.59.60.61.62.63.64.65.66.67.68.69.70.71.72.73.74.75.76.77.78.79.80.81.82.83.84.85.86.87.88.89.90.91.92.93.94.95.96.97.98.99.100.101.102.103.104.105.106.107.108.109.110.111.112.113.114.115.1593.8085887432098


In [33]:
with open('piq_pos_significant', 'wb') as fp:
    pickle.dump(pos_significant, fp)

with open('piq_neg_significant', 'wb') as fp:
    pickle.dump(neg_significant, fp)

with open('piq_pos_pvals', 'wb') as fp:
    pickle.dump(pos_pvals, fp)

with open('piq_neg_pvals', 'wb') as fp:
    pickle.dump(neg_pvals, fp)